In [17]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import messagebox

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Step 1: Read the folder
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            img = cv2.imread(os.path.join(folder_path, filename))
            if img is not None:
                images.append((filename, img))
    return images

if __name__ == "__main__":
    folder_path = "C:\\Users\\ravik\\Desktop\\My images\\test"
    
    # Step 1: Read the folder
    images = load_images_from_folder(folder_path)    

# Step 2: Process images and collect face data
def detect_faces(images):
    face_data = []
    #face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    face_cascade = cv2.CascadeClassifier('C:\\Users\\ravik\\Documents\\GitHub\\opencv\\data\\haarcascades\\haarcascade_frontalface_default.xml')
    tempnum = 0
    for filename, img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]    
            resized_face = cv2.resize(face, (64, 64))  # Define new_width and new_height
            tempnum = tempnum+1
            faceId = "face"+str(tempnum)
            face_data.append((faceId, filename, resized_face))
    return face_data
# Step 2: Process images and collect face data
face_data = detect_faces(images)
# Create a DataFrame to store face data
df = pd.DataFrame(data={
    'FaceId': [faceId for faceId, _, _ in face_data],
    'Filename': [filename for _, filename, _ in face_data],
    'Face': [face for _, _, face in face_data],
    'Consider': [0] * len(face_data)
})
df
    
current_image_index = 0
filename = ""
user_choices = {}
def show_image(face_data):
    global current_image_index, user_choices, filename
    #filename, faces = face_data
    # Initialize variables
    current_image_index = 0

    # Function to show the next image
    def show_next_image(face_data):
        global current_image_index
        if current_image_index < len(face_data):
            face = face_data[current_image_index][2]
            load_and_display_image(face)
            current_image_index += 1
        else:
            messagebox.showinfo("Finished","All images are done!")
            root.destroy()

    # Function to load and display an image
    def load_and_display_image(face):
        image1 = Image.fromarray(face)
        image2 = image1.resize((300, 300))  # Resize the image to fit the window
        photo = ImageTk.PhotoImage(image2)
        image_label.config(image=photo)
        image_label.image = photo  # Keep a reference to prevent garbage collection

    # Function to handle the "Yes" button
    def answer_yes():
        faceId = face_data[current_image_index-1][0]
        user_choices[faceId] = 1
        show_next_image(face_data)

    # Function to handle the "No" button
    def answer_no():
        faceId = face_data[current_image_index-1][0]
        user_choices[faceId] = 0
        show_next_image(face_data)

    # Create the main window
    root = tk.Tk()
    root.title("Image Viewer")

    # Create a label to display the images
    image_label = tk.Label(root)
    image_label.pack()

    # Create "Yes" and "No" buttons
    yes_button = tk.Button(root, text="Yes", command=answer_yes)
    no_button = tk.Button(root, text="No", command=answer_no)
    yes_button.pack(side=tk.LEFT)
    no_button.pack(side=tk.RIGHT)

    # Show the first image
    show_next_image(face_data)

    # Start the Tkinter main loop
    root.mainloop()

show_image(face_data)

# Step 4: Append user choices to the existing dataframe
def append_user_choices(dataframe, user_choices):
    for faceId in user_choices:
        dataframe.loc[dataframe['FaceId'] == faceId, 'Consider'] = user_choices[faceId]
    return dataframe

# Step 4: Append user choices to the existing dataframe
df = append_user_choices(df, user_choices)


In [23]:
df["Face"][1]

array([[254, 253, 250, ..., 190, 191, 192],
       [ 52,  95,  79, ..., 191, 192, 192],
       [ 58,  91,  72, ..., 194, 194, 193],
       ...,
       [254, 251, 250, ..., 255, 255, 255],
       [252, 251, 250, ..., 255, 255, 255],
       [252, 251, 251, ..., 255, 255, 255]], dtype=uint8)

In [21]:
from pathlib import Path  
filepath = Path('data/out1.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True) 
df.to_csv(filepath)

In [24]:
import numpy as np
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.set_random_seed(2019)

In [26]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (64,64,1)) ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
    tf.keras.layers.Dropout(0.1,seed = 2019),
    tf.keras.layers.Dense(400,activation ="relu"),
    tf.keras.layers.Dropout(0.3,seed = 2019),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.4,seed = 2019),
    tf.keras.layers.Dense(200,activation ="relu"),
    tf.keras.layers.Dropout(0.2,seed = 2019),
    tf.keras.layers.Dense(5,activation = "softmax")   #Adding the Output Layer
])


In [28]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
adam=Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])

In [ ]:
bs=30         #Setting batch size
train_dir = "D:/Data Science/Image Datasets/FastFood/train/"   #Setting training directory
validation_dir = "D:/Data Science/Image Datasets/FastFood/test/"   #Setting testing directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
# Flow training images in batches of 20 using train_datagen generator
#Flow_from_directory function lets the classifier directly identify the labels from the name of the directories the image lies in
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=bs,class_mode='categorical',target_size=(180,180))
# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(180,180))